### Statistical Tests - Regression

In this workbook, we will focus on validating the predictive power of selected features, which will then be used for classification/regression model
<br>

Based on the exploratory analysis, the following features were identified:
- month
- dayofweek
- holiday vs normal day
- product category
- shop id

On top of that, we will add **average monthly price** to test (assuming item_cnt will be inversely proportional to price of items)

Give the test set for the submission is asking predictions at monthly level, dayofweek and holiday features would not be applicable. We left with month, product category and shop id and average monthly price

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from datetime import datetime as dt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
sns.set()
%matplotlib inline 

C:\Users\sit_k\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
raw_df = pd.read_csv('data/train_df.csv', parse_dates=['date'])
# outlier removal steps as identified in exploratory analysis
id_out = [11, 14, 15, 12]
raw_df.loc[(raw_df.date == '2013-11-29') & (raw_df.item_category_id.isin(id_out)), 'sales'] = np.nan
raw_df.loc[(raw_df.date_block_num == 10) & (raw_df.item_category_id.isin(id_out))].interpolate(method='linear');

#### Convert daily dataframe info monthly

In [3]:
raw_df['month'] = raw_df['date'].dt.month

In [4]:
month_df = raw_df.groupby(['date_block_num', 'shop_id', 'item_category_id','month']).agg({'item_cnt_day':'sum','item_price':'mean'}).reset_index()

In [5]:
month_df_flat = pd.get_dummies(columns=['shop_id', 'item_category_id', 'month'], drop_first=True, data=month_df)

In [6]:
month_df_flat.head()

,date_block_num,item_cnt_day,item_price,shop_id_1,shop_id_2,shop_id_3,shop_id_4,shop_id_5,shop_id_6,shop_id_7,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,0,53.0,1938.688889,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,28.0,242.000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,16.0,671.357143,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,28.0,855.653846,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,65.0,1573.490909,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X = month_df_flat.iloc[:,2:]
y = month_df_flat['item_cnt_day']

In [8]:
m = sm.OLS(y, X).fit()
m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           item_cnt_day   R-squared (uncentered):                   0.554
Model:                            OLS   Adj. R-squared (uncentered):              0.553
Method:                 Least Squares   F-statistic:                              502.6
Date:                Fri, 10 Apr 2020   Prob (F-statistic):                        0.00
Time:                        20:25:51   Log-Likelihood:                     -3.7729e+05
No. Observations:               62345   AIC:                                  7.549e+05
Df Residuals:                   62191   BIC:                                  7.563e+05
Df Model:                         154                                                  
Covariance Type:            nonrobust                                                  
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
item_price             -0.0017      0.001     -2.043      0.041      -0.003   -6.72e-05
shop_id_1             -50.5064     15.899     -3.177      0.001     -81.668     -19.344
shop_id_2            -111.3798     11.086    -10.047      0.000    -133.108     -89.652
shop_id_3            -113.5816     11.089    -10.243      0.000    -135.315     -91.848
shop_id_4             -99.6824     11.054     -9.018      0.000    -121.349     -78.016
shop_id_5             -99.9047     11.076     -9.020      0.000    -121.613     -78.196
shop_id_6             -61.6136     10.999     -5.602      0.000     -83.172     -40.055
shop_id_7             -82.5435     11.048     -7.471      0.000    -104.198     -60.889
shop_id_8            -103.0438     14.347     -7.182      0.000    -131.165     -74.923
shop_id_9              99.3767     16.433      6.047      0.000      67.168     131.585
shop_id_10           -117.6736     11.106    -10.595      0.000    -139.442     -95.905
shop_id_11           -125.9408     20.642     -6.101      0.000    -166.399     -85.482
shop_id_12            -88.4283     11.056     -7.998      0.000    -110.099     -66.758
shop_id_13           -118.7036     11.709    -10.137      0.000    -141.654     -95.753
shop_id_14            -97.9556     11.061     -8.856      0.000    -119.634     -76.277
shop_id_15            -78.9424     11.074     -7.129      0.000    -100.648     -57.237
shop_id_16            -86.4798     11.025     -7.844      0.000    -108.089     -64.870
shop_id_17           -102.1661     11.326     -9.020      0.000    -124.366     -79.966
shop_id_18            -83.4036     11.027     -7.563      0.000    -105.017     -61.790
shop_id_19            -77.9475     11.034     -7.064      0.000     -99.574     -56.321
shop_id_20            229.4055     26.568      8.635      0.000     177.333     281.478
shop_id_21            -81.3554     11.021     -7.382      0.000    -102.957     -59.753
shop_id_22            -87.7316     11.032     -7.952      0.000    -109.355     -66.108
shop_id_23            -75.0906     13.714     -5.475      0.000    -101.970     -48.211
shop_id_24            -84.1860     11.059     -7.613      0.000    -105.861     -62.511
shop_id_25             26.9582     10.986      2.454      0.014       5.425      48.491
shop_id_26            -81.8129     11.032     -7.416      0.000    -103.435     -60.190
shop_id_27            -27.6158     11.044     -2.501      0.012     -49.262      -5.970
shop_id_28             -5.9018     11.007     -0.536      0.592     -27.475      15.671
shop_id_29            -81.5003     11.111     -7.335      0.000    -103.279     -59.722
shop_id_30            -74.8440     11.108     -6.738      0.000     -96.616     

**Observations**: Item prices and majority of the categorical value under shop, item_category, and month have low p-value and are significant. These feature help explain > 50% of the item_cnt variance. Adding the seasonality nature of the data as identified [before](https://github.com/sittingman/sales_forecast/blob/master/model_ts.ipynb), we can create lag features to improve the predictive power of the regression/classification models.